In [1]:
import json
from linkedin_api import Linkedin
import datetime
import pytz
import sqlite3

In [2]:
USERNAME = "chenshux@usc.edu"
PASSWORD = "jaywalker-diary-dried"
PUBLIC_PROFILE_ID = "chenshu-xu"
PROFILE_ID = "ACoAABdMtAUB1ORayXH2URJzEym1lJ8_M34crTA"

linkedin = Linkedin(USERNAME, PASSWORD, debug=True)

DEBUG:linkedin_api.client:Attempting to use cached cookies
DEBUG:linkedin_api.client:Using cached cookies
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.linkedin.com:443
DEBUG:urllib3.connectionpool:https://www.linkedin.com:443 "GET / HTTP/1.1" 200 None


In [3]:
jobs = linkedin.search_jobs(keywords="software engineer", limit=5)
with open("jobs_5.json", "w") as f:
    json.dump(jobs, f, indent=4)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.linkedin.com:443
DEBUG:urllib3.connectionpool:https://www.linkedin.com:443 "GET /voyager/api/voyagerJobsDashJobCards?decorationId=com.linkedin.voyager.dash.deco.jobs.search.JobSearchCardsCollection-174&count=5&q=jobSearch&query=(origin:JOB_SEARCH_PAGE_QUERY_EXPANSION,keywords:softwareengineer,selectedFilters:(timePostedRange:List(r86400)),spellCorrectionEnabled:true)&start=0 HTTP/1.1" 200 6858


In [4]:
job_details = []

for job in jobs:
    job_id = job['entityUrn'].split(':')[-1] # "trackingUrn": "urn:li:jobPosting:3648431448"
    job_detail = {
        "jobId": job_id,
        "status": "fail"
    }
    try:
        job_detail_data = linkedin.get_job(job_id)
        
        company_details = job_detail_data["companyDetails"]["com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany"]["companyResolutionResult"]
        
        job_url = f"https://www.linkedin.com/jobs/view/{job_id}"
        
        listed_time = datetime.datetime.fromtimestamp(job_detail_data["listedAt"]/1000)
        pdt_tz = pytz.timezone('America/Los_Angeles')
        pdt_dt = pdt_tz.localize(listed_time)
        pdt_str = pdt_dt.strftime('%Y-%m-%d %H:%M:%S %Z')
        
        job_detail.update({
            "status": "success",
            "jobUrl": job_url,
            "companyDetails": {
                "companyId": company_details["entityUrn"],
                "companyName": company_details["name"],
                "companyUrl": company_details["url"]
            },
            "description": job_detail_data["description"]["text"],
            "title": job_detail_data["title"],
            "postTime": pdt_str,
            "postTimeStamp": listed_time.timestamp()
        })
    finally:
        job_details.append(job_detail)
        
with open("jobDetails_5.json", "w") as f:
    json.dump(job_details, f, indent=4)

DEBUG:urllib3.connectionpool:https://www.linkedin.com:443 "GET /voyager/api/jobs/jobPostings/3705615609?decorationId=com.linkedin.voyager.deco.jobs.web.shared.WebLightJobPosting-23 HTTP/1.1" 200 5460
DEBUG:urllib3.connectionpool:https://www.linkedin.com:443 "GET /voyager/api/jobs/jobPostings/3713337625?decorationId=com.linkedin.voyager.deco.jobs.web.shared.WebLightJobPosting-23 HTTP/1.1" 200 3481
DEBUG:urllib3.connectionpool:https://www.linkedin.com:443 "GET /voyager/api/jobs/jobPostings/3705611377?decorationId=com.linkedin.voyager.deco.jobs.web.shared.WebLightJobPosting-23 HTTP/1.1" 200 4497
DEBUG:urllib3.connectionpool:https://www.linkedin.com:443 "GET /voyager/api/jobs/jobPostings/3675174189?decorationId=com.linkedin.voyager.deco.jobs.web.shared.WebLightJobPosting-23 HTTP/1.1" 200 2976
DEBUG:urllib3.connectionpool:https://www.linkedin.com:443 "GET /voyager/api/jobs/jobPostings/3713120642?decorationId=com.linkedin.voyager.deco.jobs.web.shared.WebLightJobPosting-23 HTTP/1.1" 200 3651


In [5]:
connection = sqlite3.connect('mydatabase.db')
cursor = connection.cursor()

In [6]:
cursor.execute(
    '''
    create table if not exists company
    (
        companyId   TEXT not null,
        companyName TEXT,
        companyUrl  TEXT,
        id          INTEGER
            constraint company_pk
                primary key autoincrement,
        constraint company_pk2
            unique (companyId, id)
    )
    '''
)
connection.commit()

In [7]:
cursor.execute(
    '''
    create table if not exists jobPost
    (
        jobId         TEXT not null,
        jobUrl        TEXT,
        title         TEXT,
        companyId     INTEGER
            constraint jobPost_companyId_fk
                references company,
        description   TEXT,
        postTime      TEXT,
        postTimeStamp INTEGER,
        status        TEXT,
        count         integer,
        id            INTEGER
            constraint jobPost_pk
                primary key autoincrement,
        constraint jobPost_pk2
            unique (jobId, id)
    );
    '''
)
connection.commit()

In [8]:
cursor.execute(
    '''
    create table if not exists log
    (
        id        INTEGER
            constraint log_pk
                primary key autoincrement,
        timeStamp INTEGER,
        jobPostId INTEGER not null
            constraint log_pk2
                unique
            constraint log_jobPost_id_fk
                references jobPost,
        message   TEXT
    );
    '''
)
connection.commit()

In [17]:
for job_detail in job_details:
    company_details = job_detail["companyDetails"]
    # if company not exit, add to company table
    cursor.execute("SELECT id FROM company WHERE companyId = ?;", (company_details["companyId"],))
    row = cursor.fetchone()
    if row is None:
        cursor.execute("INSERT INTO company (companyId, companyName, companyUrl) VALUES (?, ?, ?);",
                       (company_details["companyId"], 
                        company_details["companyName"], 
                        company_details["companyUrl"]))
        company_row_id = cursor.lastrowid
    else:
        company_row_id = int(row[0])
    connection.commit()
    
    cursor.execute('''
        insert into jobPost (jobId, jobUrl, title, companyId, description, postTime, postTimeStamp, status) values (?,?,?,?,?,?,?,?);
    ''', (
            job_detail["jobId"], 
            job_detail["jobUrl"], 
            job_detail["title"], 
            company_row_id, 
            job_detail["description"], 
            job_detail["postTime"], 
            job_detail["postTimeStamp"], 
            job_detail["status"]
        )
   )
    connection.commit()
        